In [1]:
import numpy as np
from scipy import linalg
from scipy.optimize import minimize

In [2]:
#                 (0,244)--------(294,244)
#                     |               |
# Polygone ->         |               |
#                     |               |
#                   (0,0)--------(294, 0)

Zhang’s Calibration DLT METHOD

In [3]:
pol1 = (554/1280, 237/720)
pol2 = (343/1280, 490/720)
pol3 = (934/1280, 693/720)
pol4 = (978/1280, 278/720)
pol1_2 = (461/1280, 349/720)
pol2_2 = (583/1280, 572/720)
pol3_2 = (961/1280, 438/720)
pol4_2 = (727/1280, 254/720)

# gps_points = [(54.725242, 55.940438),#0
#               (54.725207, 55.940667),#1
#               (54.725185, 55.940805),#2

#               (54.725309, 55.940874),#3
#               (54.725433, 55.940939),#4
#               (54.725458, 55.940803),#5

#               (54.725500, 55.940560),#6
#               (54.725367, 55.940505)]#7


gps_points = [(54.72525477847287, 55.94040626769686),#0
              (54.72521765988856, 55.940605807425584),#1
              (54.7251860813648, 55.94077177075803),#2

              (54.72531516535042, 55.9408465981561),#3
              (54.72543981687801, 55.94090703567065),#4
              (54.725471949209265, 55.94073243840754),#5

              (54.72550463552051, 55.940547288563664),#6
              (54.72538464609278, 55.94048557093376)]#7

x_res, y_res = (1920, 1080)# resolution
polygon = np.array([(pol1[0] * x_res, pol1[1] * y_res), 
                    (pol1_2[0] * x_res, pol1_2[1] * y_res), 
                    (pol2[0] * x_res, pol2[1] * y_res),
                    (pol2_2[0] * x_res, pol2_2[1] * y_res),
                    (pol3[0] * x_res, pol3[1] * y_res),
                    (pol3_2[0] * x_res, pol3_2[1] * y_res),
                    (pol4[0] * x_res, pol4[1] * y_res), 
                    (pol4_2[0] * x_res, pol4_2[1] * y_res)], dtype=int)#points for polygone in the center

def zang(pt):
    
    # pointsW = [(0, 244), (0,122), (0, 0), (147, 0), (294, 0), (294, 122), (294, 244), (147, 244)]
    pointsW = gps_points
    pointsPX = polygon
    print(pointsW, '\n', pointsPX)

    if len(pointsPX) != len(pointsW): return

    A = np.zeros((len(pointsW)*2, 9))
    for k, W, PX in zip(range(0, len(pointsW)*2, 2),pointsW, pointsPX):
        xw = W[0]
        yw = W[1]

        u = PX[0]
        v = PX[1]

        A[k] = [-xw, -yw, -1, 0, 0, 0, u*xw, u*yw, u]
        A[k+1] = [0, 0, 0, -xw, -yw, -1, v*xw, v*yw, v]
    
    U,D,V = np.linalg.svd(A)
    m_ext = np.reshape(V[-1] / V[-1][-1], (3,3))
    vector = np.array([pt[0], pt[1], 1])
    print(m_ext)
    #BACKWARD PIXEL -> WORLD
    result = np.linalg.solve(m_ext, vector)
    result = result / result[-1]
    print(result[:2])

    #FORWARD WORLD -> PIXEL
    # result = np.dot(m_ext, vector)
    # result = result / result[-1]
    # print(result)

zang([1146, 435])

[(54.72525477847287, 55.94040626769686), (54.72521765988856, 55.940605807425584), (54.7251860813648, 55.94077177075803), (54.72531516535042, 55.9408465981561), (54.72543981687801, 55.94090703567065), (54.725471949209265, 55.94073243840754), (54.72550463552051, 55.940547288563664), (54.72538464609278, 55.94048557093376)] 
 [[ 831  355]
 [ 691  523]
 [ 514  735]
 [ 874  858]
 [1401 1039]
 [1441  657]
 [1467  417]
 [1090  381]]
[[ 2.10302973e+01 -1.22791129e+01 -4.63979104e+02]
 [-4.87218236e+00  5.07541375e+00 -1.72847276e+01]
 [-6.45725593e-03 -1.15589441e-02  1.00000000e+00]]
[54.72539747 55.94054384]
